In [1]:
from gensim.summarization import summarize
from newspaper import Article
from model import Model
from utils import build_dict, build_dataset, batch_iter
from rouge import Rouge
import pickle
import pandas as pd
import re
import tensorflow as tf
import random

import nltk 
from nltk.tag import pos_tag
from nltk import FreqDist
from nltk.tokenize import word_tokenize

import sym_vocab
import ant_vocab

nltk.download('averaged_perceptron_tagger')

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer


# import tensorflow as tf
import nltk, re, pprint
import docx2txt
import numpy as np
import os

import ant_vocab


# download 
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package words to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# 지문 생성 및 문제 생성을 위한 텍스트 파일 생성 
def crawl_news(url, raw_txt_name, num_words):
    # URL 기반 크롤링, 언어는 영어로 설정
    news = Article(url, language = 'en') # URL 가져오기
    news.download() # 해당 url의 html을 가져오기
    news.parse() # html 파싱
    
    print(news.text)
    # 지정된 단어 갯수를 기준으로 textrank를 적용하여 파싱된 텍스트를 요약 (Rule base)
    text = summarize(news.text, word_count = num_words) # 600자 기준으로 하기
    title = news.title # 파싱한 html에서 title에 해당하는 부분 추출하기
    title = title.lower() # title에 대해서 전부 소문자화 
    
    title = re.sub('\([^)]*\)', '', title) # 괄호를 포함한 문자열 제거 
    
    # 크롤링한 raw data를 별도로 저장
    with open(raw_txt_name, 'w', encoding = 'utf-8') as f: # 크롤링한 raw data를 저장하기
        f.write(text)
    
    text = re.sub('\n', ' ', text) # raw data에서 불필요한 줄바꿈표 제거 (상대적일듯)

    return text, title

In [3]:
# 빈칸 지문 생성을 위한 함수를 별도로 정의
def bf_create_order(raw_txt_name, prepro_txt_name):
    # 문장 번호: 문장 형식의 저장을 위해 변수 생성
    test_dict = {}
    
    # 저장했었던 크롤링 본문 데이터 불러오기
    with open(raw_txt_name, 'r', encoding = 'utf-8') as f:
    # raw data 상태에서 전처리를 통해 모델에 들어갈 수 있는 데이터셋으로 변형
        for idx, text in enumerate(f.readlines()):
            # print(idx, text)
            #text = text.lower() # 소문자로 통일
            text = re.sub('[.]', '', text) # 일단 온점 모두 제거
            #text = re.sub('[,]', ' ,', text) # 반점 하나의 단어로 인식
            #text = re.sub("\’s",' \'s', text) # 소유격도 하나의 단어로 인식
            #text = re.sub("[0-9]", '#', text) # 숫자를 #로 치환
            #text = re.sub("'.+'", '', text) # 작은 따옴표 안 고유명사 제거 
            #text = re.sub('[“”]', '', text) # 큰 따옴표 제거 
            text = re.sub('\n', '', text) # 줄 바꿈표 제거 
            print(idx, text)
        
            # 사전에 저장하기
            test_dict[idx] = text ## {1: '텍스트', 2: '텍스트'}
            
    # 전처리된 데이터를 별도의 파일로 저장
    with open(prepro_txt_name, 'w', encoding = 'utf-8') as f: # 전처리한 데이터를 저장하기 
        # 줄 바꿈을 기준으로 문장을 구분하여 저장
        for i in range(len(test_dict)):
            test_dict[i] = test_dict[i] + " " + "." + '\n'
            f.write(test_dict[i])
    
    return test_dict 

In [4]:
# 텍스트 요약 학습 데이터 불러오기
def shake_bot_order(filename, vaild_article_path):
    # 설정된 세팅 값 불러오기
    with open("args.pickle", "rb") as f:
        args = pickle.load(f)
        
    # 실행시 최초의 상태로 초기화하여 모델 재사용이 가능하도록 함
    tf.reset_default_graph()
    # 학습된 단어 사전과 본문 최대 길이와 요약 최대 길이 설정 값을 불러옴
    print("셰봇이 단어 사전 불러오는 중...")
    word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("valid", args.toy)
    # 불러온 설정 값에 따라서 검증 데이터셋을 생성
    print("셰봇이 기본 설정 값 초기화 중...")
    valid_x = build_dataset("valid", vaild_article_path, word_dict, article_max_len, summary_max_len, args.toy)
    valid_x_len = [len([y for y in x if y != 0]) for x in valid_x]

    with tf.Session() as sess:
        # 학습된 체크포인트와 모델을 불러오기 
        print("Loading saved model...")
        model = Model(reversed_dict, article_max_len, summary_max_len, args, forward_only=True) # 검증 단계
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state("./saved_model/")
        saver.restore(sess, ckpt.model_checkpoint_path)
        
        # 검증을 진행할 배치 사이즈 설정
        batches = batch_iter(valid_x, [0] * len(valid_x), args.batch_size, 1)
        
        # 검증 단계인 만큼 인코더 인풋 부분만 고려하여 학습 진행
        print("셰봇이 수능 순서 문제 생성 준비 중.. {}...".format(filename))
        for batch_x, _ in batches:
            # 문장 -> 문장 예측이 가능하도록 데이터 분할 
            batch_x_len = [len([y for y in x if y != 0]) for x in batch_x]
            # 검증을 위한 배치에 따라 모델 및 데이터 셋 설정 준비 
            valid_feed_dict = {
                model.batch_size: len(batch_x),
                model.X: batch_x,
                model.X_len: batch_x_len,
            }
            # Prediction을 목적으로 학습된 모델을 검증을 위한 설정 값을 적용하여 예측 결과 생성
            prediction = sess.run(model.prediction, feed_dict=valid_feed_dict)
            # 인덱스 -> 단어 사전을 통해 학습된 결과를 단어로 표현.
            prediction_output = [[reversed_dict[y] for y in x] for x in prediction[:, 0, :]]
            
            # 생성된 예측 결과물을 문장 단위로 저장 
            with open(filename, "w") as f:
                for line in prediction_output:
                    summary = list()
                    for word in line:
                        if word == "</s>":
                            break
                        if word not in summary:
                            summary.append(word)
                    print(" ".join(summary), file=f)
                    
            #with open('./rouge_test/sys_dir/{}.txt'.format(filename), "w") as f:
                #for line in prediction_output:
                    #summary = list()
                    #for word in line:
                        #if word == "</s>":
                            #break
                        #if word not in summary:
                            #summary.append(word)
                    #print(" ".join(summary), file=f)

        print('셰봇이가 5지 선다로 쓸만한 친구들을 추려냈어요.. {}...'.format(filename))

In [5]:
def make_order_sentence(test_dict):
    
    # 제시 문장 
    main_set = test_dict[0]
    
    # 몇개의 문장씩 나눌 것인지 결정
    divide_num = (int(len(test_dict)-1)) // 3
    remain_num = (int(len(test_dict)-1)) % 3
    
    if divide_num >= remain_num * 2:
        # divide_num이 2개 이므로 1을 빼면 2를 뺀 효과
        divide_num = int(divide_num - 1) 
        # 따라서 remain_num에 +2를 해야함. 
        remain_num = int(remain_num + 2)
    
    # 랜덤하게 배정하기 위한 리스트 형성
    assign_list = [int(divide_num), int(divide_num), int(remain_num)]
    random.shuffle(assign_list)
    
    # 배정받은 수만큼 문장 나누기
    first_set = list()
    second_set = list()
    third_set = list()
    
    # 첫번째 할당
    for n in range(1, assign_list[0] + 1):
        first_set.append(test_dict[n].capitalize())
        
    # 두번째 할당
    for n in range(assign_list[0] + 1, assign_list[0] + assign_list[1] + 1):
        second_set.append(test_dict[n].capitalize())
        
    # 세번째 할당
    for n in range(assign_list[0] + assign_list[1] + 1, assign_list[0] + assign_list[1] + assign_list[2] + 1):
        third_set.append(test_dict[n].capitalize())
    
    return main_set, first_set, second_set, third_set # 아직 배열 되어 있지 않은 상황. 

In [6]:
def make_order_sentence2(test_dict):
    
    # 제시 문장 
    main_set = test_dict[0]
    
    # 몇개의 문장씩 나눌 것인지 결정
    
    divide_num = (int(len(test_dict)-1)) // 5 # 5개의 분할

    remain_num = (int(len(test_dict)-1)) % 5 # 5개의 분할 중 마지막 남은 것 (크게 차이가 없을 것)
    
    # 랜덤하게 배정하기 위한 리스트 형성
    # 나머지가 존재하는 경우
    if remain_num != 0:
    
        assign_list = [int(divide_num), int(divide_num), int(divide_num), int(divide_num), int(remain_num)]
    
        random.shuffle(assign_list)
    # 나머지가 존재하지 않고 나누어 떨어지는 경우 
    elif remain_num == 0:
        
        assign_list = [int(divide_num), int(divide_num), int(divide_num), int(divide_num), int(divide_num)]
    
        random.shuffle(assign_list)
    
    # 배정받은 수만큼 문장 나누기
    first_set = list()
    second_set = list()
    third_set = list()
    fourth_set = list()
    fifth_set = list()
    
    # 첫번째 할당
    for n in range(1, assign_list[0] + 1):
        first_set.append(test_dict[n].capitalize())
    # 두번째 할당
    for n in range(assign_list[0] + 1, assign_list[0] + assign_list[1] + 1):
        second_set.append(test_dict[n].capitalize())
    # 세번째 할당
    for n in range(assign_list[0] + assign_list[1] + 1, assign_list[0] + assign_list[1] + assign_list[2] + 1):
        third_set.append(test_dict[n].capitalize())
    # 네번째 할당
    for n in range(assign_list[0] + assign_list[1] + assign_list[2] + 1, assign_list[0] + assign_list[1] + assign_list[2] + assign_list[3] + 1):
        fourth_set.append(test_dict[n].capitalize())
    # 다섯번째 할당 
    for n in range(assign_list[0] + assign_list[1] + assign_list[2] + assign_list[3] + 1, assign_list[0] + assign_list[1] + assign_list[2] + assign_list[3] + assign_list[4] + 1):
        fifth_set.append(test_dict[n].capitalize())
    
    return main_set, first_set, second_set, third_set, fourth_set, fifth_set # 아직 배열 되어 있지 않은 상황. 

In [7]:
# Rouge Score를 통해 학습 완성도가 높은 문장을 선별 
def for_rouge_test(summy, label):
    rouge_dict = {}
    rouge_list = []
    rouge = Rouge()
    # 학습 결과로 예측한 결과물 불러오기    
    with open(summy, 'r', encoding = 'utf-8') as f:
            
        for idx, line in enumerate(f.readlines()):
            
            # 실제 문장 - 예측 결과에서 학습의 정도를 Rouge-r과 Rouge-p로 검증
            scores = rouge.get_scores(line, label[idx])
            
            p_score = scores[0]["rouge-1"]["p"] 
            r_score = scores[0]["rouge-1"]["r"] 
            
            if r_score > 0:
            
                r_idx_score = (idx, r_score)            
                
                rouge_list.append(r_idx_score)
                # print(rouge_list)
            
                rouge_dict[idx] = line
            
            
        rouge_list = sorted(rouge_list, key = lambda rouge_list: rouge_list[1], reverse=True)
        
        rouge_list = rouge_list[:5]
        
        sort_rouge_list = sorted(rouge_list, key = lambda rouge_list: rouge_list[0], reverse=False)
                
        question_list = []
    
        for ix in sort_rouge_list:
            
            question_list.append(rouge_dict[ix[0]])
          
    return question_list, sort_rouge_list

In [8]:
# 일치하지 않는 문장 선택하는 문제 생성 
url = 'https://www.nytimes.com/2019/11/14/us/politics/on-politics-deval-patrick-mike-bloomberg.html'
raw_txt_name = 'CSAT_35.txt'
prepro_txt_name = 'prepro_35.txt'
learned_txt_name = 'order_35.txt'
num = 1 # 정답으로 할 선지 선택 
num_words = 400
def order_35(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기 
    text, title = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_order(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # Rule Base, 5개 문장 할당하기 
    main_set, first_set, second_set, third_set, fourth_set, fifth_set = make_order_sentence2(test_dict)
    
    # main set 가공하기
    main_set = main_set.capitalize()
    main_set = re.sub(' [.]', '.', main_set)
    main_set = re.sub(' ,', ',', main_set)
    main_set = re.sub('\n', '', main_set)

    # question set 가공하기
    question_list = [first_set, second_set, third_set, fourth_set, fifth_set]
    alpha_list = ['①', '②', '③', '④', '⑤']
    
    # 생성된 선지를 그대로 사용하지 않고, 전처리 작업 수행 
    for ix, final_question in enumerate(question_list):
        
        final_question = alpha_list[ix] + ' ' + ' '.join(final_question)
        
        final_question = re.sub(' [.]', '.', final_question)
        
        final_question = re.sub(' ,', ',', final_question)
        
        final_question = re.sub('\n', '', final_question)
        # 5개 선지 리스트 만들기 
        question_list[ix] = final_question
    
    # 5개 선지 중 설정한 번호의 선지를 정답으로 만들기
    answer_sent = question_list[num-1]
    print(answer_sent)
    
    # 정답 선지 만들기
    token = word_tokenize(answer_sent)
    for word in token:
        # 정답을 만들기 위해 반의어 사전 활용
        para = ant_vocab.crawling_dict(word)
        # 반의어가 존재하는 경우에 대입
        if len(para) > 0:
            
            token[token.index(word)] = para
            
    # 넘버링 제외하고 첫 글자는 대문자화         
    token[1] = token[1].capitalize()
    answer_sent = ' '.join(token)
    answer_sent = re.sub(' [.]', '.', answer_sent)
    answer_sent = re.sub(' ,', ',', answer_sent)
    
    
    # 본문 교체
    question_list[num-1] = answer_sent
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('35. 다음 글에서 전체 흐름과 관계 없는 문장은?')
    print('-----------------------------------')
    print(main_set, end = ' ')
    for question in question_list:
        print(str(question), end = ' ')
    
if __name__ == "__main__":
    
    order_35(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num)

People who’ve spoken to Mr. Patrick say he sees an opportunity in the flagging poll numbers of Senators Kamala Harris and Cory Booker, the two major black candidates already in the race. If he can do decently well in early-voting New Hampshire, his neighboring state, he may gain support in South Carolina, where the majority of the primary electorate is black, the thinking goes.

The obstacles to this plan are obvious: money and manpower. Having been out of public life for nearly five years, Mr. Patrick has no list of donors to tap. And, at such a late date, he’s scrambling to assemble a team, pulling in former staff members of Beto O’Rourke.

Money isn’t a problem for Mr. Bloomberg, whose net worth is estimated to be more than $52 billion. But the first Iowa poll to include Mr. Bloomberg since news broke of his possible bid shows him with a negative 31 percent approval rating, making him the least popular candidate in the field. In New Hampshire, 54 percent of likely primary voters alr

In [9]:
# A-B-C 순서 맞추기 
url = 'https://www.nytimes.com/2019/11/13/reader-center/trust-news-sources.html'
raw_txt_name = 'CSAT_36.txt'
prepro_txt_name = 'prepro_36.txt'
learned_txt_name = 'order_36.txt'
num = 1

def order_36(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words,num):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기
    text, title = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_order(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 4개 문장 추출 
    main_set, first_set, second_set, third_set = make_order_sentence(test_dict)
    
    # main set 가공하기
    main_set = main_set.capitalize()
    main_set = re.sub(' [.]', '.', main_set)
    main_set = re.sub(' ,', ',', main_set)
    main_set = re.sub('\n', '', main_set)

    # question set 가공하기
    question_list = [first_set, second_set, third_set]
    
    answer_list = [first_set, second_set, third_set]
    
    alpha_list = ['(A)', '(B)', '(C)']
    
    random.shuffle(question_list) # 선지를 랜덤하게 섞기
    
    # 생성된 선지를 그대로 사용하지 않고, 전처리 작업 수행
    for ix, final_question in enumerate(question_list):
        
        final_question = alpha_list[ix] + ' ' + ' '.join(final_question)
        
        final_question = re.sub(' [.]', '.', final_question)
        
        final_question = re.sub(' ,', ',', final_question)
        
        final_question = re.sub('\n', '', final_question)
        
        question_list[ix] = final_question
        
    
    # 정답 배열 만들기 
    for ix, ans in enumerate(answer_list):
        
        ans = ' '.join(ans)
        
        ans = re.sub(' [.]', '.', ans)
        
        ans = re.sub(' ,', ',', ans)
        
        ans = re.sub('\n', '', ans)
        
        print( '{} 번째는 {}'.format(ix+1, ans)) 
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('36. 주어진 글 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.')
    print('-----------------------------------')
    print(main_set)
    print('-----------------------------------')
    for question in question_list:
        print(str(question))
    
if __name__ == "__main__":
    
    order_36(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num)

We are interested in how you get information and how you feel that has changed.

It used to be that Walter Cronkite r ead the news every night, and Americans tuned in and went to bed with the same set of facts, even if they had different political views.

But that news environment is gone.

These days there is a sea of information — a lot of it conflicting — and people are left to sort through it. Some people still have one trusted source. Others are choosing to become their own curators, taking in a variety of news sources before deciding whom and what to believe .

This is coming at a moment of rising political polarization, when political parties are ever more divided, and the ideological lines have hardened .

I’m a reporter on our national desk and I write stories about the political and cultural divide in America. I have taken special interest in people who do not line up with one side or the other — people in the middle who tell me they feel politically homeless.
0 We are intere

In [10]:
url = 'https://www.nytimes.com/2019/11/14/world/europe/germany-measles-vaccine.html'
raw_txt_name = 'CSAT_37.txt'
prepro_txt_name = 'prepro_37.txt'
learned_txt_name = 'order_37.txt'
num = 1
num_words = 500

def order_37(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기
    text, title = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_order(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 4개 문장 추출 
    main_set, first_set, second_set, third_set = make_order_sentence(test_dict)
    
    # main set 가공하기
    main_set = main_set.capitalize()
    main_set = re.sub(' [.]', '.', main_set)
    main_set = re.sub(' ,', ',', main_set)
    main_set = re.sub('\n', '', main_set)

    # question set 가공하기
    question_list = [first_set, second_set, third_set]
    
    answer_list = [first_set, second_set, third_set]
    
    alpha_list = ['(A)', '(B)', '(C)']
    
    random.shuffle(question_list) # 선지를 랜덤하게 섞기
    
    # 생성된 선지를 그대로 사용하지 않고, 전처리 작업 수행
    for ix, final_question in enumerate(question_list):
        
        final_question = alpha_list[ix] + ' ' + ' '.join(final_question)
        
        final_question = re.sub(' [.]', '.', final_question)
        
        final_question = re.sub(' ,', ',', final_question)
        
        final_question = re.sub('\n', '', final_question)
        
        question_list[ix] = final_question
        
    
    # 정답 배열 만들기 
    for ix, ans in enumerate(answer_list):
        
        ans = ' '.join(ans)
        
        ans = re.sub(' [.]', '.', ans)
        
        ans = re.sub(' ,', ',', ans)
        
        ans = re.sub('\n', '', ans)
        
        print( '{} 번째는 {}'.format(ix+1, ans))  
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('37. 주어진 글 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.')
    print('-----------------------------------')
    print(main_set)
    print('-----------------------------------')
    for question in question_list:
        print(str(question))
    
if __name__ == "__main__":
    
    order_37(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num)

BERLIN — Parents in Germany must vaccinate their children against measles or face fines of several thousand euros under a law passed on Thursday that aims to stop the spread of a disease that has returned in recent years after decades of decline.

The law, which is to take effect from March next year , will require all children seeking to attend preschool to prove that they have been immunized or risk losing their placement. Children aged 6 and older, who are required by law to attend school, must also show proof of having received a vaccine.

Health officials in Germany have recorded 501 cases of measles this year, despite the existence of vaccines that are proven to be safe and effective. According to the World Health Organization, outbreaks of the disease have increased in recent years across the globe, including in developed countries where it had largely been eradicated.

In an address to Parliament before the vote in favor of the law, Jens Spahn, Germany’s health minister, called

In [11]:
# 순서에 적합한 문장 삽입하기 
url = 'https://www.nytimes.com/2019/11/14/technology/apple-harvard-health-studies.html'
raw_txt_name = 'CSAT_38.txt'
prepro_txt_name = 'prepro_38.txt'
learned_txt_name = 'order_38.txt'
num = 1 # 정답으로 할 선지 선택 
num_words = 500
def order_38(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기
    text, title = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_order(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임.  
    # 5개 분할로 문장 추출
    main_set, first_set, second_set, third_set, fourth_set, fifth_set = make_order_sentence2(test_dict)
    
    # main set 가공하기
    main_set = main_set.capitalize()
    main_set = re.sub(' [.]', '.', main_set)
    main_set = re.sub(' ,', ',', main_set)
    main_set = re.sub('\n', '', main_set)
    
    print(main_set)
     
    question_list = [first_set, second_set, third_set, fourth_set, fifth_set]
    alpha_list = ['( ① )', '( ② )', '( ③ )', '( ④ )', '( ⑤ )']

    # answer_sent 가공하기
    answer_sent = question_list[num-1]
    
    answer_sent = re.sub('\n', '', answer_sent[0])
    answer_sent = re.sub(' \.', '.', answer_sent)
    print(answer_sent) # 삽입해야 하는 문장 

    # question set 가공하기 
    if num == 1:
        
        question_list[num-1] = main_set
    
    else:

        question_list.pop(num-1)
        
        question_list.insert(0, main_set)
        
    # 생성된 선지에 문장 삽입이 가능한 형태로 변형
    for ix, final_question in enumerate(question_list):
        
        # 첫번째 main_set의 고정적으로 (1) 문장 번호를 부여 
        if ix == 0: 
            
            final_question = final_question + ' ' + alpha_list[ix]
        # 나머지의 경우, 정답 문장을 제외하고 문장 번호 부여 
        else:
            
            final_question = ' '.join(final_question) + ' ' + alpha_list[ix]
        
            final_question = re.sub(' [.]', '.', final_question)
        
            final_question = re.sub(' ,', ',', final_question)
        
            final_question = re.sub('\n', '', final_question)         
        
        question_list[ix] = final_question
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('38. 글의 흐름으로 보아, 주어진 문장이 들어가기에 가장 적절한 곳을 고르시오.')
    print('-----------------------------------')
    print(answer_sent)
    print('-----------------------------------')
    print(main_set, end = ' ')
    for question in question_list:
        print(str(question), end = ' ')
    
if __name__ == "__main__":
    
    order_38(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num)

In 1976, the Harvard School of Public Health and two other major medical institutions started a study on nurses that has become one of the largest and longest research efforts ever conducted on women’s health. They have so far enrolled more than 275,000 participants.

On Thursday, the Harvard school announced an even more ambitious women’s health study, one that aims to enroll a million women over a decade.

The new ingredients allowing the huge scale: Apple’s iPhones, apps and money.

Harvard’s new study is just one of three new large research efforts that Apple is working on with leading academic research centers an d health organization s. Together, the studies, which Apple is paying for, show how the Silicon Valley giant and its popular products are reshaping medical research.

To enroll in clinical trials, patients have often had to travel to medical centers to be briefed by researchers and fill out the study paperwork in person. Many studies also follow patients only intermittent

In [12]:
url = 'https://www.nytimes.com/2019/11/15/us/politics/deval-patrick-michael-bloomberg-2020.html?action=click&module=Top%20Stories&pgtype=Homepage'
raw_txt_name = 'CSAT_39.txt'
prepro_txt_name = 'prepro_39.txt'
learned_txt_name = 'order_39.txt'
num = 5 # 정답으로 할 선지 선택 
num_words = 500
def order_39(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기
    text, title = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_order(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임.  
    # 5개 분할로 문장 추출
    main_set, first_set, second_set, third_set, fourth_set, fifth_set = make_order_sentence2(test_dict)
    
    # main set 가공하기
    main_set = main_set.capitalize()
    main_set = re.sub(' [.]', '.', main_set)
    main_set = re.sub(' ,', ',', main_set)
    main_set = re.sub('\n', '', main_set)
    
    print(main_set)
     
    question_list = [first_set, second_set, third_set, fourth_set, fifth_set]
    alpha_list = ['( ① )', '( ② )', '( ③ )', '( ④ )', '( ⑤ )']

    # answer_sent 가공하기
    answer_sent = question_list[num-1]
    
    answer_sent = re.sub('\n', '', answer_sent[0])
    answer_sent = re.sub(' \.', '.', answer_sent)
    print(answer_sent) # 삽입해야 하는 문장 

    # question set 가공하기 
    if num == 1:
        
        question_list[num-1] = main_set
    
    else:

        question_list.pop(num-1)
        
        question_list.insert(0, main_set)
        
    # 생성된 선지에 문장 삽입이 가능한 형태로 변형
    for ix, final_question in enumerate(question_list):
        
        # 첫번째 main_set의 고정적으로 (1) 문장 번호를 부여 
        if ix == 0: 
            
            final_question = final_question + ' ' + alpha_list[ix]
        # 나머지의 경우, 정답 문장을 제외하고 문장 번호 부여 
        else:
            
            final_question = ' '.join(final_question) + ' ' + alpha_list[ix]
        
            final_question = re.sub(' [.]', '.', final_question)
        
            final_question = re.sub(' ,', ',', final_question)
        
            final_question = re.sub('\n', '', final_question)         
        
        question_list[ix] = final_question
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('39. 글의 흐름으로 보아, 주어진 문장이 들어가기에 가장 적절한 곳을 고르시오.')
    print('-----------------------------------')
    print(answer_sent)
    print('-----------------------------------')
    print(main_set, end = ' ')
    for question in question_list:
        print(str(question), end = ' ')
    
if __name__ == "__main__":
    
    order_39(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num)

It remains to be seen whether the two men will inspire and animate a powerful moderate coalition in a way that Mr. Biden has not, or splinter the party’s less-liberal constituencies and effectively strengthen Ms. Warren or Mr. Sanders. Or, they might just find themselves entirely superfluous in an already crowded field.

“We have several moderates in the campaign now, but they’ve not been able to fill the lane,” said Marlon Kimpson, a state senator in South Carolina who has been hosting town hall-style events with presidential candidates. “Now, you’ve got all these people wanting to dip their toe in the water and test the likelihood that they can fill this role.”

Mr. Kimpson said Mr. Biden was still the most natural option for moderate voters, but warned that voters “don’t seem to be very motivated about his candidacy.” He said he was skeptical that either Mr. Bloomberg or Mr. Patrick could do better with such a delayed start, though he considered Mr. Patrick as by far the more intrig